<a href="https://colab.research.google.com/github/amnesiac14/IA/blob/main/Tarea_4_Red_Neuronal_Multicapa_Christopher_Maya_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Red neuronal multicapa

Esta red neuronal tiene la intención de predecir la forma farmacéutica de un medicamento a partir de su descripción original. Esto pensado en una primera parte del problema de clasificar medicamentos y devolver su clave de cuadro básico a partir de su descripción original; para la cual, necesitamos clasificar por etapas sus diferentes diferenciadores: molecula, forma farmaceutica, concentracion y presentacion

Sin embargo, el primer problema a superar es la homologación correcta de los medicamentos, la interpretacion de texto correcta de descripciones originales demasiado sucias que incluso pueden interferir en la vectorizacion de texto, situacion que debe considerarse antes de hacer el entrenamiento y realizar de forma eficiente las etiquetas en cada bloque.

Se decidió este tema por tener un problema de esta naturaleza en la realidad, sin embargo, no tuve el tiempo de realizarlo de mejor manera, este modelo estará muy propenso a over u underfitting.

Quise usar este ejmplo para poder ir enfocando este problema real e ir investigando a la par como poder resolverlo mientras trataba de entender mejor a las redes neuronales multicapa

In [2]:
cuadro_basico = pd.read_csv("/content/Cuadro Basico de medicamentos.csv")
cuadro_basico.head()

,CLAVE,DESCRIPCION,TIPO DE INSUMO,MOLECULA,FORMA FARMACEUTICA,GRUPO FORMA FARMACEUTICA,CONCENTRACION,PRESENTACION
0,030.000.0003.00,SUCEDANEO DE LECHE HUMANA DE PRETERMINO POLVO ...,LACTEOS,SUCEDANEO DE LECHE HUMANA DE PRETERMINO,POLVO,SOLIDOS,KCAL 400-525; LIPIDOS 19.2-31.5G; RELAC A. LIN...,ENVASE CON 450 A 454 G Y MEDIDA DE 4.40 A 4.50 G
1,030.000.0011.00,SUCEDANEO DE LECHE HUMANA DE TERMINO POLVO KCA...,LACTEOS,SUCEDANEO DE LECHE HUMANA DE TERMINO,POLVO,SOLIDOS,"KCAL 509-528/100G, LIPIDOS 25.80-28.90/100G, P...",ENVASE DESDE 360 G HASTA 454 G POLVO Y MEDIDA ...
2,030.000.0011.01,SUCEDANEO DE LECHE HUMANA DE TERMINO POLVO KCA...,LACTEOS,SUCEDANEO DE LECHE HUMANA DE TERMINO,POLVO,SOLIDOS,"KCAL 509-528/100G, LIPIDOS 25.80-28.90/100G, P...",ENVASE DESDE 59 ML HASTA 237 ML.
3,030.000.0012.00,SUCEDANEO DE LECHE HUMANA DE TERMINO SIN LACTO...,LACTEOS,SUCEDANEO DE LECHE HUMANA DE TERMINO SIN LACTOSA,POLVO,SOLIDOS,"KCAL 502-522/100G, LIPIDOS 25.0-28.0/100G, PRO...",ENVASE CON 400 A 454 G Y MEDIDA DE 4.30 A 4.5 G
4,030.000.0013.00,FORMULA DE PROTEINA EXTENSAMENTE HIDROLIZADA P...,LACTEOS,FORMULA DE PROTEINA EXTENSAMENTE HIDROLIZADA,POLVO,SOLIDOS,"KCAL 100/100G, LIPIDOS 4.4-6/100G, PROTEINAS 2...",ENVASE DESDE 357 G HASTA 450 G POLVO Y MEDIDA ...


In [1]:
!pip install -U tensorflow spacy
!python -m spacy download es_core_news_sm

import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout


# Cargar spaCy español para quitar stopwords y lematizar, que el procesamiento de texto sea mas sencillo

nlp = spacy.load("es_core_news_sm")
stopwords = nlp.Defaults.stop_words


def preprocesar(texto):
    doc = nlp(str(texto))
    tokens = [token.lemma_.lower() for token in doc if token.is_alpha and token.text.lower() not in stopwords]
    return " ".join(tokens)

# Aplicar preprocesamiento a la columna DESCRIPCION
cuadro_basico['descripcion_procesada'] = cuadro_basico['DESCRIPCION'].apply(preprocesar)

# Preparar etiquetas

le = LabelEncoder()
y = le.fit_transform(cuadro_basico['FORMA FARMACEUTICA'])

# tokenizar y convertir en secuencias de numeros y aplicar padding para tener la misma longitud
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cuadro_basico['descripcion_procesada']) #hace un abiyeccion del conjunto de palabras en este corpus a los naturales
X_seq = tokenizer.texts_to_sequences(cuadro_basico['descripcion_procesada'])
X_pad = pad_sequences(X_seq, padding='post') #rellena con ceros para tener la misma longitud, entonces tenemos un indice extra aqui por el 0

# Dividir conjunto de datos para entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

# Red neuronal multicapa
vocab_size = len(tokenizer.word_index) + 1  #aqui sumamos el indice extra para tener la dimension correcta

model = Sequential([
    # esta capa tiene entrada a cada indice de cada palabra en el corpus y su salida es la conversion a vectores en R^50
    Embedding(input_dim=vocab_size, output_dim=50, input_length=X_pad.shape[1]),
    Flatten(),  # Convertimos embeddings 2D en 1D para Dense, es decir, en un vector de 50*input_length entradas que entra a Dense
    Dense(128, activation='relu'),
    Dropout(0.3), #profundizare mas en este porque investigue que esto apaga el 30% de las neuronas aleatoriamente para eviar el overfitting
    Dense(64, activation='relu'),
    Dense(len(le.classes_), activation='softmax')  # Salida multicategoría de forma farmaceutica
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # compilacion del modelo


# Entrenamiento
history = model.fit(X_train, y_train, validation_split=0.1, epochs=20, batch_size=16)

# Evaluacion
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy en test: {acc:.4f}")

# Prediccion
ejemplo = "TABLETA ACIDO ACETILSALICILICO 500 MG"
proceso = preprocesar(ejemplo)
seq = tokenizer.texts_to_sequences([proceso])
pad = pad_sequences(seq, maxlen=X_pad.shape[1], padding='post')
pred = model.predict(pad)
pred_clase = le.inverse_transform([pred.argmax()])[0]
print(f"Predicción: {pred_clase}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 106.6 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.19.0
    Uninstalling tensorflow-2.19.0:
      Successfully uninstalled tensorflow-2.19.0
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.8
    Uninstalling spacy-3.8.8:
      Successfully uninstalled spacy-3.8.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.20.0 which

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 19s 34ms/step - accuracy: 0.3780 - loss: 2.9304 - val_accuracy: 0.6707 - val_loss: 1.4571
Epoch 2/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.7042 - loss: 1.3441 - val_accuracy: 0.8233 - val_loss: 0.8423
Epoch 3/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - accuracy: 0.8327 - loss: 0.7474 - val_accuracy: 0.8810 - val_loss: 0.6277
Epoch 4/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - accuracy: 0.8771 - loss: 0.5425 - val_accuracy: 0.9195 - val_loss: 0.5542
Epoch 5/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.9186 - loss: 0.3503 - val_accuracy: 0.9267 - val_loss: 0.5071
Epoch 6/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.9275 - loss: 0.2861 - val_accuracy: 0.9411 - val_loss: 0.5151
Epoch 7/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9419 - loss: 0.2368 - val_accuracy: 0.9495 - val_loss: 0.5018
Epoch 8/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.9464 - loss: 0.1923 - va